In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
import copy
# from lmphoton.simulation import current_simulation as sim
import pandas as pd
from itertools import product
from tqdm import tqdm
from copy import deepcopy
import xarray as xr

import sys
sys.path.append("../")

# SPPD optical elements
from src.sppd_clm_optics import SPPDCLMOPTICS_LAMBDA_AB

# PMIC drivers
from src.pikeriver_pmic import PMIC

# Calibration and control
from src.sppd_clm import CONTROLLER
from src.laser_control.pid import pid as PID

plt.style.use("plot_style.mplstyle")
warnings.filterwarnings("ignore")

In [2]:
enablence_mux_bw_lambda = np.array([20])/17*0.1e-9
enablence_mux_3sigma = np.array([17])/17*0.1e-9
sppd_laser_3sigma = np.array([38])/17*0.1e-9
no_mux = np.array([0])
wpe_slope_3sigma = np.array([0.15])
lin_sweep_val_range = np.array([0.1,0.2,0.3])
mpd_current_clamp_range = np.array([40e-3])


no_lasers = 16
no_iterations = 1001

In [3]:
enablence_mux_mc = np.zeros((no_iterations, no_lasers, len(enablence_mux_3sigma)))
sppd_laser_mc = np.zeros((no_iterations, no_lasers, len(sppd_laser_3sigma)))
wpe_slope_factor_mc = np.zeros((no_iterations, no_lasers, len(wpe_slope_3sigma)))

for idd_iter in range(no_iterations):
  for idd_eb in range(len(enablence_mux_3sigma)):
      enablence_mux_mc[idd_iter, :, idd_eb] = np.random.normal(0, enablence_mux_3sigma[idd_eb]/3) * np.ones(no_lasers)
  for idd_laser in range(no_lasers):
    for idd_sppd in range(len(sppd_laser_3sigma)):
      sppd_laser_mc[idd_iter, idd_laser, idd_sppd] = np.random.normal(0, sppd_laser_3sigma[idd_sppd]/3)
    for idd_wpe in range(len(wpe_slope_3sigma)):
      wpe_slope_factor_mc[idd_iter, idd_laser, idd_wpe] = np.random.normal(1.0, wpe_slope_3sigma[idd_wpe]/3)

In [4]:
target_grid_array = np.array([
  1301.47, 1302.60, 1303.73, 1304.87,
  1306.01, 1307.14, 1308.28, 1309.43,
  1310.57, 1311.72, 1312.87, 1314.02,
  1315.17, 1316.33, 1317.48, 1318.64
  ])*1e-9 

wavelength_fabrication = np.array([
  1301.47, 1302.60, 1303.73, 1304.87,
  1306.01, 1307.14, 1308.28, 1309.43,
  1310.57, 1311.72, 1312.87, 1314.02,
  1315.17, 1316.33, 1317.48, 1318.64
  ])*1e-9

In [5]:
data_statistics_lists = []

for idd_mux, idd_iter, idd_eb, idd_sppd, idd_bw_lambda, idd_wpe, idd_lin_sweep in tqdm(product(range(len(no_mux)), range(no_iterations), range(len(enablence_mux_3sigma)), range(len(sppd_laser_3sigma)), range(len(enablence_mux_bw_lambda)), range(len(wpe_slope_3sigma)), range(len(lin_sweep_val_range))), total=no_iterations*len(enablence_mux_bw_lambda)*len(sppd_laser_3sigma)*len(enablence_mux_3sigma)*len(no_mux)*len(wpe_slope_3sigma)*len(lin_sweep_val_range)):
  enablence_grid = target_grid_array + enablence_mux_mc[idd_iter, :, idd_eb]
  sppd_grid = target_grid_array + sppd_laser_mc[idd_iter, :, idd_sppd] - 0.5e-9
  wpe_slope_factor = wpe_slope_factor_mc[idd_iter, :, idd_wpe]

  # Create the SPPD DFB laser
  _sppd_clm_optics_mc = SPPDCLMOPTICS_LAMBDA_AB(
    current = 0.133,
    wavelength_fabrication=sppd_grid,
    target_grid_array=enablence_grid,
    bw_lambda = enablence_mux_bw_lambda[idd_bw_lambda],
    nominal_junction_temperature=273+44.5,
    wpe_slope_factor = wpe_slope_factor,
    name = "SPPDCLMOPTICS_AB_2",
    no_mux = idd_mux + 1
  )

  _sppd_clm_optics_mc.update(tc = 273+42.5, i_array=np.ones(16) * 0.133)

  _pk_pmic = PMIC()

  dt = 10e-3
  _pid_tec = pid_tec = PID(kp=0.5, ki=10*dt, kd=10*dt, dt=dt, umax=3, umin=-3)
  _pid_idrive = [PID(kp=1/1000, ki=1/1000, kd=1/1000, dt=dt, umax = 1024, umin=-1024) for __ in range(len(wavelength_fabrication))]

  controller = CONTROLLER(
    sppd_clm_optics = _sppd_clm_optics_mc,
    pmic = _pk_pmic,
    pid_tec = _pid_tec,
    pid_idrive = _pid_idrive,
    dt = dt,
    drive_current_array = np.ones(16) * 0.133,
  )

  controller._tend = 15
  controller._tdither = 5
  controller._tagg_1 = 14.0
  controller._tagg_2 = 14.5
  controller._agg_1_amp = 0
  controller._agg_2_amp = 0

  controller.calibrate(
    tc=273+35,
    temperature_array = np.linspace(30, 50, 201)+273, 
    drive_current_array= np.linspace(0.1, 0.2, 101),
    power_wavelength_calibration=True,
    tec_temperature_tuning=True,
    drive_current_tuning=False,
  )

  controller.control(
    lin_sweep_range=lin_sweep_val_range[idd_lin_sweep],
    output_power_clamp_bool = False, 
    output_power_clamp = 0.04
  )
  
  controller_statistics = controller._time_step_statistics
  idd_value = 0
  for key, values in controller_statistics.items():
    idd_value_2 = 0
    idd_value += 1
    for key_2, values_2 in values.items():
      idd_value_2 += 1
      data_statistics_lists += [controller_statistics[key][key_2].tolist()]


100%|██████████| 3003/3003 [18:02:16<00:00, 21.62s/it]  


In [6]:
data_statistics_array = np.array(deepcopy(data_statistics_lists)).reshape((len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), len(lin_sweep_val_range), 5, 4*4, no_lasers))

In [ ]:
data_export = xr.Dataset(
  data_vars = dict(
    enablence_mux_mc = (["idd_iter", "idd_laser", "idd_eb"], enablence_mux_mc),
    sppd_laser_mc = (["idd_iter", "idd_laser", "idd_sppd"], sppd_laser_mc),
    wpe_slope_factor_mc = (["idd_iter", "idd_laser", "idd_wpe"], wpe_slope_factor_mc),
    data_statistics_array = (["idd_mux", "idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "lin_sweep_range", "controller_observable", "observable_stat", "idd_laser"], data_statistics_array)
    ),
  coords = dict(
    idd_mux = no_mux,
    idd_iter = range(no_iterations),
    idd_eb = range(len(enablence_mux_3sigma)),
    idd_sppd = range(len(sppd_laser_3sigma)),
    idd_bw_lambda = range(len(enablence_mux_bw_lambda)),
    idd_laser = range(no_lasers),
    idd_wpe = range(len(wpe_slope_3sigma)),
    lin_sweep_range = lin_sweep_val_range,
    controller_observable = ["drive_current", "enablence_output_power", "enablence_output_lambda", "sppd_pout", "sppd_laser_junction_temperature"],
    observable_stat = ["std", "mean", " max", "min","std_no_agg", "mean_no_agg", "max_no_agg", "min_no_agg", "std_agg_1", "mean_agg_1", "max_agg_1", "min_agg_1", "std_agg_2", "mean_agg_2", "max_agg_2", "min_agg_2"], 
  ),
  attrs=dict(description = "SPPD CLM Controller Monte Carlo Simulation simulations - dated: 2024-04-24")
)

data_export.to_netcdf("../simulation_data/sppd_clm_controller_1_mux_tec_tuning_output_power_clamp_false_mc_20240430_v0p1.nc")